#### DeepStream 8.0 - Multi-Class Object Detection with Tracking & Attribute Classification

This notebook demonstrates a comprehensive DeepStream pipeline for:
- **Primary object detection** (vehicles, persons, bicycles, road signs)
- **Object tracking** across frames
- **Secondary inference** for vehicle attributes (color, make, type)
- **On-Screen Display** with attribute overlay


**Pipeline Flow:**
```
filesrc → h264parse → nvv4l2decoder → nvstreammux → 
pgie (primary detector) → nvtracker → 
sgie1 (car type) → sgie2 (car make) → 
nvvideoconvert → nvdsosd → nvvideoconvert → capsfilter → 
nvv4l2h264enc → h264parse → mp4mux → filesink
```


In [1]:
import sys
import time

sys.path.append('/opt/nvidia/deepstream/deepstream-8.0/sources/deepstream_python_apps/apps')

import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst, GLib
from common.bus_call import bus_call
import pyds

Gst.init(None)

[]

In [2]:
PGIE_CLASS_ID_VEHICLE = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_PERSON = 2
PGIE_CLASS_ID_ROADSIGN = 3

INPUT_VIDEO_NAME = '/opt/nvidia/deepstream/deepstream-8.0/samples/streams/sample_720p.h264'
OUTPUT_VIDEO_NAME = '/app/notebooks/ds_out_multiclass.mp4'

PGIE_CONFIG_FILE = '/app/notebooks/dstest2_pgie_config.txt'
SGIE1_CONFIG_FILE = '/app/notebooks/dstest2_sgie1_config.txt'
SGIE2_CONFIG_FILE = '/app/notebooks/dstest2_sgie2_config.txt'

TRACKER_LIB = '/opt/nvidia/deepstream/deepstream-8.0/lib/libnvds_nvmultiobjecttracker.so'

print(f" Input video: {INPUT_VIDEO_NAME}")
print(f" Output video: {OUTPUT_VIDEO_NAME}")
print(f" Primary GIE config: {PGIE_CONFIG_FILE}")
print(f" SGIE1 config (Make): {SGIE1_CONFIG_FILE}")
print(f" SGIE2 config (Type): {SGIE2_CONFIG_FILE}")
print(f" Tracker library: {TRACKER_LIB}")

 Input video: /opt/nvidia/deepstream/deepstream-8.0/samples/streams/sample_720p.h264
 Output video: /app/notebooks/ds_out_multiclass.mp4
 Primary GIE config: /app/notebooks/dstest2_pgie_config.txt
 SGIE1 config (Make): /app/notebooks/dstest2_sgie1_config.txt
 SGIE2 config (Type): /app/notebooks/dstest2_sgie2_config.txt
 Tracker library: /opt/nvidia/deepstream/deepstream-8.0/lib/libnvds_nvmultiobjecttracker.so


In [3]:
def make_elm_or_print_err(factoryname, name, printedname, detail=""):
    """Create a GStreamer element or print error message"""
    print(f"Creating {printedname}...")
    elm = Gst.ElementFactory.make(factoryname, name)
    if not elm:
        sys.stderr.write(f"Unable to create {printedname}\n")
        if detail:
            sys.stderr.write(detail)
    return elm

print(" Helper function defined")

 Helper function defined


In [4]:
print("\n" + "="*60)
print("CREATING PIPELINE")
print("="*60)
pipeline = Gst.Pipeline()
if not pipeline:
    sys.stderr.write("Unable to create Pipeline\n")
source = make_elm_or_print_err("filesrc", "file-source", "Source")
h264parser = make_elm_or_print_err("h264parse", "h264-parser", "H264 Parser")
decoder = make_elm_or_print_err("nvv4l2decoder", "nvv4l2-decoder", "NV Decoder")
streammux = make_elm_or_print_err("nvstreammux", "stream-muxer", "Stream Muxer")
pgie = make_elm_or_print_err("nvinfer", "primary-inference", "Primary Inference")
tracker = make_elm_or_print_err("nvtracker", "tracker", "NV Tracker")

sgie1 = make_elm_or_print_err("nvinfer", "secondary1-nvinference-engine", "Secondary Inference 1 (Make)")
sgie2 = make_elm_or_print_err("nvinfer", "secondary2-nvinference-engine", "Secondary Inference 2 (Type)")

nvvidconv = make_elm_or_print_err("nvvideoconvert", "convertor", "NV Video Converter 1")
nvosd = make_elm_or_print_err("nvdsosd", "onscreendisplay", "On-Screen Display")
nvvidconv2 = make_elm_or_print_err("nvvideoconvert", "convertor2", "NV Video Converter 2")
capsfilter = make_elm_or_print_err("capsfilter", "caps", "Caps Filter")
encoder = make_elm_or_print_err("nvv4l2h264enc", "encoder", "H264 Encoder")
h264parser2 = make_elm_or_print_err("h264parse", "h264-parser2", "H264 Parser 2")
mp4mux = make_elm_or_print_err("mp4mux", "mp4mux", "MP4 Muxer")
sink = make_elm_or_print_err("filesink", "filesink", "File Sink")

print("\n All elements created successfully!")


CREATING PIPELINE
Creating Source...
Creating H264 Parser...
Creating NV Decoder...
Creating Stream Muxer...
Creating Primary Inference...
Creating NV Tracker...
Creating Secondary Inference 1 (Make)...
Creating Secondary Inference 2 (Type)...
Creating NV Video Converter 1...
Creating On-Screen Display...
Creating NV Video Converter 2...
Creating Caps Filter...
Creating H264 Encoder...
Creating H264 Parser 2...
Creating MP4 Muxer...
Creating File Sink...

 All elements created successfully!


/bin/bash: line 1: lsmod: command not found


In [5]:
print("\n" + "="*60)
print("CONFIGURING ELEMENTS")
print("="*60)

source.set_property('location', INPUT_VIDEO_NAME)
print(f"Source location: {INPUT_VIDEO_NAME}")

streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', 1)
streammux.set_property('batched-push-timeout', 4000000)
print("Stream muxer: 1920x1080, batch-size=1")

pgie.set_property('config-file-path', PGIE_CONFIG_FILE)
print(f"Primary GIE config: {PGIE_CONFIG_FILE}")

tracker.set_property('tracker-width', 640)
tracker.set_property('tracker-height', 384)
tracker.set_property('ll-lib-file', TRACKER_LIB)
tracker.set_property('ll-config-file', '/opt/nvidia/deepstream/deepstream-8.0/samples/configs/deepstream-app/config_tracker_NvDCF_perf.yml')
tracker.set_property('gpu-id', 0)
print("Tracker configured: NvDCF")

sgie1.set_property('config-file-path', SGIE1_CONFIG_FILE)
print(f"SGIE1 config (Make): {SGIE1_CONFIG_FILE}")

sgie2.set_property('config-file-path', SGIE2_CONFIG_FILE)
print(f"SGIE2 config (Type): {SGIE2_CONFIG_FILE}")

caps = Gst.Caps.from_string("video/x-raw(memory:NVMM), format=I420")
capsfilter.set_property("caps", caps)
print("Caps filter: I420 format")

encoder.set_property('bitrate', 4000000)
print("Encoder bitrate: 4 Mbps")

sink.set_property('location', OUTPUT_VIDEO_NAME)
sink.set_property('sync', False)
print(f"Output file: {OUTPUT_VIDEO_NAME}")


CONFIGURING ELEMENTS
Source location: /opt/nvidia/deepstream/deepstream-8.0/samples/streams/sample_720p.h264
Stream muxer: 1920x1080, batch-size=1
Primary GIE config: /app/notebooks/dstest2_pgie_config.txt
Tracker configured: NvDCF
SGIE1 config (Make): /app/notebooks/dstest2_sgie1_config.txt
SGIE2 config (Type): /app/notebooks/dstest2_sgie2_config.txt
Caps filter: I420 format
Encoder bitrate: 4 Mbps
Output file: /app/notebooks/ds_out_multiclass.mp4


In [6]:
print("\n" + "="*60)
print("BUILDING PIPELINE")
print("="*60)

print("Adding elements to pipeline...")
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(tracker)
pipeline.add(sgie1)
pipeline.add(sgie2)
pipeline.add(nvvidconv)
pipeline.add(nvosd)
pipeline.add(nvvidconv2)
pipeline.add(capsfilter)
pipeline.add(encoder)
pipeline.add(h264parser2)
pipeline.add(mp4mux)
pipeline.add(sink)
print("All elements added")

print("\nLinking elements...")
source.link(h264parser)
h264parser.link(decoder)

sinkpad = streammux.get_request_pad("sink_0")
if not sinkpad:
    sys.stderr.write("Unable to get sink pad of streammux\n")
srcpad = decoder.get_static_pad("src")
if not srcpad:
    sys.stderr.write("Unable to get source pad of decoder\n")
srcpad.link(sinkpad)
streammux.link(pgie)
pgie.link(tracker)
tracker.link(sgie1)
sgie1.link(sgie2)
sgie2.link(nvvidconv)
nvvidconv.link(nvosd)
nvosd.link(nvvidconv2)
nvvidconv2.link(capsfilter)
capsfilter.link(encoder)
encoder.link(h264parser2)
h264parser2.link(mp4mux)
mp4mux.link(sink)

print("All elements linked")
print("\nPipeline built successfully")


BUILDING PIPELINE
Adding elements to pipeline...
All elements added

Linking elements...
All elements linked

Pipeline built successfully


/tmp/ipykernel_6818/2728493970.py:28: DeprecationWarning: Gst.Element.get_request_pad is deprecated
  sinkpad = streammux.get_request_pad("sink_0")


In [7]:
def osd_sink_pad_buffer_probe(pad, info, u_data):
    """Callback function to process metadata from each frame including secondary classifiers"""
    
    obj_counter = {
        PGIE_CLASS_ID_VEHICLE: 0,
        PGIE_CLASS_ID_PERSON: 0,
        PGIE_CLASS_ID_BICYCLE: 0,
        PGIE_CLASS_ID_ROADSIGN: 0
    }
    
    frame_number = 0
    num_rects = 0
    
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer")
        return Gst.PadProbeReturn.OK

    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    
    while l_frame is not None:
        try:
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        
        frame_number = frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj = frame_meta.obj_meta_list
        
        while l_obj is not None:
            try:
                obj_meta = pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            
            obj_counter[obj_meta.class_id] += 1
            
            if obj_meta.class_id == PGIE_CLASS_ID_VEHICLE:
                vehicle_color = "Unknown"
                vehicle_make_type = "Unknown"
                
                l_classifier = obj_meta.classifier_meta_list
                while l_classifier is not None:
                    try:
                        classifier_meta = pyds.NvDsClassifierMeta.cast(l_classifier.data)
                        l_label = classifier_meta.label_info_list
                        
                        while l_label is not None:
                            try:
                                label_info = pyds.NvDsLabelInfo.cast(l_label.data)
                                
                                if classifier_meta.unique_component_id == 2: 
                                    vehicle_color = label_info.result_label
                                elif classifier_meta.unique_component_id == 3:
                                    vehicle_make_type = label_info.result_label
                                
                                l_label = l_label.next
                            except StopIteration:
                                break
                        
                        l_classifier = l_classifier.next
                    except StopIteration:
                        break
                
                if vehicle_color != "Unknown" or vehicle_make_type != "Unknown":
                    display_meta = pyds.nvds_acquire_display_meta_from_pool(batch_meta)
                    display_meta.num_labels = 1
                    text_params = display_meta.text_params[0]
                    
                    attr_text = f"ID:{obj_meta.object_id} | {vehicle_color} | {vehicle_make_type}"
                    text_params.display_text = attr_text
                    
                    text_params.x_offset = int(obj_meta.rect_params.left)
                    text_params.y_offset = int(obj_meta.rect_params.top) - 30
                    
                    text_params.font_params.font_name = "Serif"
                    text_params.font_params.font_size = 10
                    text_params.font_params.font_color.set(1.0, 1.0, 0.0, 1.0)
                    text_params.set_bg_clr = 1
                    text_params.text_bg_clr.set(0.0, 0.0, 0.0, 0.7)
                    
                    pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
                
                print(f" Vehicle ID:{obj_meta.object_id} - Color:{vehicle_color}, Make/Type:{vehicle_make_type}")
            
            try:
                l_obj = l_obj.next
            except StopIteration:
                break
        
        display_meta = pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        
        py_nvosd_text_params.display_text = "Frame={} Objects={} Vehicles={} Persons={}".format(
            frame_number, num_rects, 
            obj_counter[PGIE_CLASS_ID_VEHICLE], 
            obj_counter[PGIE_CLASS_ID_PERSON]
        )
        
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 10
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)
        py_nvosd_text_params.set_bg_clr = 1
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
        
        print(pyds.get_string(py_nvosd_text_params.display_text))
        
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        
        try:
            l_frame = l_frame.next
        except StopIteration:
            break
    
    return Gst.PadProbeReturn.OK

In [8]:
osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    sys.stderr.write("Unable to get sink pad of nvosd\n")
else:
    osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)
    print(" Metadata probe attached to OSD element")

 Metadata probe attached to OSD element


In [9]:
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

print(" Bus message handler configured")

 Bus message handler configured


In [10]:
import os
if os.path.exists(OUTPUT_VIDEO_NAME):
    os.remove(OUTPUT_VIDEO_NAME)

print("\n" + "="*60)
print("STARTING PIPELINE")
print("="*60)
print(f"Processing: {INPUT_VIDEO_NAME}")
print("="*60 + "\n")

pipeline.set_state(Gst.State.PLAYING)
start_time = time.time()

try:
    loop.run()
except Exception as e:
    print(f"Error: {e}")

elapsed = time.time() - start_time
pipeline.set_state(Gst.State.NULL)

print(f"\n{'='*60}")
print(f"PIPELINE COMPLETE")
print(f"{'='*60}")
print(f" Processing time: {elapsed:.1f} seconds")

if os.path.exists(OUTPUT_VIDEO_NAME):
    size_mb = os.path.getsize(OUTPUT_VIDEO_NAME) / (1024 * 1024)
    print(f" Output file: {OUTPUT_VIDEO_NAME} ({size_mb:.1f} MB)")
else:
    print(" WARNING: Output file not created!")


STARTING PIPELINE
Processing: /opt/nvidia/deepstream/deepstream-8.0/samples/streams/sample_720p.h264



0:00:00.336874642  6818     0x1010d310 WARN                 nvinfer gstnvinfer.cpp:682:gst_nvinfer_logger:<secondary2-nvinference-engine> NvDsInferContext[UID 3]: Warning from NvDsInferContextImpl::deserializeEngineAndBackend() <nvdsinfer_context_impl.cpp:2097> [UID = 3]: deserialize engine from file :/opt/nvidia/deepstream/deepstream-8.0/samples/models/Secondary_VehicleTypes/resnet18_vehicletypenet_pruned.onnx_b16_gpu0_fp16.engine failed
0:00:00.336890754  6818     0x1010d310 WARN                 nvinfer gstnvinfer.cpp:682:gst_nvinfer_logger:<secondary2-nvinference-engine> NvDsInferContext[UID 3]: Warning from NvDsInferContextImpl::generateBackendContext() <nvdsinfer_context_impl.cpp:2202> [UID = 3]: deserialize backend context from engine from file :/opt/nvidia/deepstream/deepstream-8.0/samples/models/Secondary_VehicleTypes/resnet18_vehicletypenet_pruned.onnx_b16_gpu0_fp16.engine failed, try rebuild
0:00:00.336894002  6818     0x1010d310 INFO                 nvinfer gstnvinfer.cpp:68

0:00:08.074802124  6818     0x1010d310 INFO                 nvinfer gstnvinfer.cpp:685:gst_nvinfer_logger:<secondary2-nvinference-engine> NvDsInferContext[UID 3]: Info from NvDsInferContextImpl::buildModel() <nvdsinfer_context_impl.cpp:2155> [UID = 3]: serialize cuda engine to file: /opt/nvidia/deepstream/deepstream-8.0/samples/models/Secondary_VehicleTypes/resnet18_vehicletypenet_pruned.onnx_b16_gpu0_fp16.engine successfully


0:00:08.416119127  6818     0x1010d310 INFO                 nvinfer gstnvinfer_impl.cpp:343:notifyLoadModelStatus:<secondary2-nvinference-engine> [UID 3]: Load new model:/app/notebooks/dstest2_sgie2_config.txt sucessfully
0:00:08.416218952  6818     0x1010d310 WARN                 nvinfer gstnvinfer.cpp:682:gst_nvinfer_logger:<secondary1-nvinference-engine> NvDsInferContext[UID 2]: Warning from NvDsInferContextImpl::deserializeEngineAndBackend() <nvdsinfer_context_impl.cpp:2097> [UID = 2]: deserialize engine from file :/opt/nvidia/deepstream/deepstream-8.0/samples/models/Secondary_VehicleMake/resnet18_vehiclemakenet_pruned.onnx_b16_gpu0_fp16.engine failed
0:00:08.416224200  6818     0x1010d310 WARN                 nvinfer gstnvinfer.cpp:682:gst_nvinfer_logger:<secondary1-nvinference-engine> NvDsInferContext[UID 2]: Warning from NvDsInferContextImpl::generateBackendContext() <nvdsinfer_context_impl.cpp:2202> [UID = 2]: deserialize backend context from engine from file :/opt/nvidia/deeps

0:00:14.711418541  6818     0x1010d310 INFO                 nvinfer gstnvinfer.cpp:685:gst_nvinfer_logger:<secondary1-nvinference-engine> NvDsInferContext[UID 2]: Info from NvDsInferContextImpl::buildModel() <nvdsinfer_context_impl.cpp:2155> [UID = 2]: serialize cuda engine to file: /opt/nvidia/deepstream/deepstream-8.0/samples/models/Secondary_VehicleMake/resnet18_vehiclemakenet_pruned.onnx_b16_gpu0_fp16.engine successfully


...

 Vehicle ID:342 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:357 - Color:Unknown, Make/Type:largevehicle
 Vehicle ID:362 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:239 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:278 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:366 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:345 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:348 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:329 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:351 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:356 - Color:mercedes, Make/Type:sedan
 Vehicle ID:334 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:256 - Color:dodge, Make/Type:truck
 Vehicle ID:328 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:365 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:359 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:272 - Color:dodge, Make/Type:largevehicle
Frame=1049 Objects=21 Vehicles=17 Persons=4
 Vehicle ID:333 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:345 - Color:Unknown, Ma

 Vehicle ID:382 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:362 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:389 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:387 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:306 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:278 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:394 - Color:lexus, Make/Type:sedan
 Vehicle ID:342 - Color:ford, Make/Type:sedan
 Vehicle ID:239 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:383 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:386 - Color:Unknown, Make/Type:largevehicle
 Vehicle ID:351 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:405 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:392 - Color:audi, Make/Type:suv
 Vehicle ID:376 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:329 - Color:Unknown, Make/Type:Unknown
Frame=1164 Objects=20 Vehicles=16 Persons=4
 Vehicle ID:345 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:382 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:389 - Color:Unknown, Make/Type:Unknow

 Vehicle ID:278 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:362 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:376 - Color:gmc, Make/Type:van
 Vehicle ID:382 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:422 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:239 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:436 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:398 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:329 - Color:mercedes, Make/Type:suv
 Vehicle ID:439 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:417 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:357 - Color:Unknown, Make/Type:largevehicle
 Vehicle ID:438 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:426 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:424 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:336 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:427 - Color:Unknown, Make/Type:sedan
Frame=1279 Objects=24 Vehicles=17 Persons=7
 Vehicle ID:278 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:418 - Color:Unknown, Make/Type:Unk

 Vehicle ID:451 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:482 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:464 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:426 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:465 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:481 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:239 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:278 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:478 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:473 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:417 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:455 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:445 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:424 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:459 - Color:honda, Make/Type:sedan
 Vehicle ID:462 - Color:Unknown, Make/Type:Unknown
 Vehicle ID:467 - Color:Unknown, Make/Type:largevehicle
 Vehicle ID:475 - Color:Unknown, Make/Type:Unknown
Frame=1400 Objects=21 Vehicles=18 Persons=3
 Vehicle ID:464 - Color:Unknown, Make


PIPELINE COMPLETE
 Processing time: 50.4 seconds
 Output file: /app/notebooks/ds_out_multiclass.mp4 (22.8 MB)
